# Анализ эксперимента neural_controller-v3

## Постановка эксперимента

Копия эксперимента neural_controller-v2 с использованием оберток и исправленной нормализации.

In [ ]:
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from nn_laser_stabilizer.config.config import load_config
from nn_laser_stabilizer.utils.paths import get_experiment_dir

In [ ]:
EXPERIMENT_NAME = "neural_controller-v3"
EXPERIMENT_DATE = "2026-02-24"
EXPERIMENT_TIME = "15-45-52"

EXPERIMENT_DIR_PATH = get_experiment_dir(
    experiment_name=EXPERIMENT_NAME, 
    experiment_date=EXPERIMENT_DATE, 
    experiment_time=EXPERIMENT_TIME)

config = load_config(EXPERIMENT_DIR_PATH / "config.yaml")
print(f"Эксперимент: {config.experiment_name}")
print(f"Окружение: {config.env.name}")
print(f"Алгоритм: {config.algorithm.type}")

## Загрузка и парсинг данных

In [ ]:
def _flatten_dict(data: dict, prefix: str = "") -> dict:
    flat = {}
    for key, value in data.items():
        full_key = f"{prefix}{key}"
        if isinstance(value, dict):
            flat.update(_flatten_dict(value, prefix=f"{full_key}."))
        else:
            flat[full_key] = value
    return flat


def _normalize_record(record: dict) -> dict:
    # Новый формат collector.jsonl: event + env_info + policy_info.
    # Для других логов (например env.jsonl exchange) оставляем запись как есть.
    if "env_info" not in record and "policy_info" not in record:
        return dict(record)

    normalized = {
        k: v for k, v in record.items()
        if k not in ("env_info", "policy_info")
    }

    env_info = record.get("env_info")
    if isinstance(env_info, dict):
        # Поля окружения оставляем плоскими: error, reward, ...
        normalized.update(_flatten_dict(env_info))

    policy_info = record.get("policy_info")
    if isinstance(policy_info, dict):
        normalized.update(_flatten_dict(policy_info, prefix="policy_"))

    return normalized


def load_jsonl(path: Path, source: str | None = None, event: str | None = None) -> pd.DataFrame:
    rows = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                record = json.loads(line)
            except json.JSONDecodeError:
                continue

            normalized = _normalize_record(record)

            if source and normalized.get("source") != source:
                continue
            if event and normalized.get("event") != event:
                continue
            rows.append(normalized)

    return pd.DataFrame(rows)

In [ ]:
step_log_path = EXPERIMENT_DIR_PATH / "collector.jsonl"
if not step_log_path.exists():
    raise FileNotFoundError(f"Не найден лог шагов: {step_log_path}")

step_df = load_jsonl(step_log_path, event="step").reset_index(drop=True)
step_df["global_step"] = step_df.index + 1

policy_cols = sorted([c for c in step_df.columns if c.startswith("policy_")])
env_cols = sorted([
    c for c in step_df.columns
    if c not in ("global_step", "event") and not c.startswith("policy_")
])

env_df = step_df[["global_step", *env_cols]].copy()
policy_df = step_df[["global_step", *policy_cols]].copy()

# Для policy-полей, которые приходят как [x], создаем scalar-версии.
def _first_scalar(value):
    if isinstance(value, (list, tuple, np.ndarray)) and len(value) > 0:
        return float(value[0])
    if isinstance(value, (int, float, np.floating, np.integer)):
        return float(value)
    return np.nan

for col in ["policy_action", "policy_mean_action", "policy_raw_action", "policy_log_prob", "policy_std"]:
    if col in policy_df.columns:
        policy_df[f"{col}_0"] = policy_df[col].apply(_first_scalar)

print(f"Файл шагов: {step_log_path.name}")
print(f"step_df: {len(step_df)} записей")
print(f"env_df: {env_df.shape[0]}x{env_df.shape[1]}")
print(f"policy_df: {policy_df.shape[0]}x{policy_df.shape[1]}")
if len(step_df) > 0:
    print(f"Диапазон global_step: {step_df['global_step'].min()} — {step_df['global_step'].max()}")

print(f"Policy-полей найдено: {len(policy_cols)}")
if policy_cols:
    print("Примеры policy-полей:", policy_cols[:10])

env_df.tail()

In [ ]:
train_df = load_jsonl(EXPERIMENT_DIR_PATH / "train.jsonl", source="train", event="step")
print(f"Шаги обучения: {len(train_df)} записей")
print(f"Диапазон шагов: {train_df['step'].min()} — {train_df['step'].max()}")
train_df.head()

In [ ]:
exploration_steps = config.exploration.get('steps', 0)
print(f"Exploration steps: {exploration_steps}")

## Анализ логов окружения

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['error'], alpha=0.8, linewidth=0.5, label='Error')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Ошибка регулирования')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Error')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['reward'], alpha=0.8, linewidth=0.5, color='tab:green', label='Reward')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Награда')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Reward')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['process_variable'], 'b-', alpha=0.7, linewidth=0.5, label='Process Variable')
plt.plot(env_df['global_step'], env_df['setpoint'], 'r--', linewidth=1.5, label='Setpoint')
plt.axvline(x=exploration_steps, color='orange', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Значение')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['control_output'], 'g-', alpha=0.7, linewidth=0.5, label='Control Output')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Управляющий сигнал')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Control Output')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(policy_df['global_step'], policy_df['policy_action_0'], alpha=0.7, linewidth=0.5, color='tab:purple', label='policy_action_0')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Действие policy (первый компонент)')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('policy_action_0')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
env_df

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['step_interval_us'], alpha=0.7, linewidth=0.5, color='tab:brown')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Интервал между шагами')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Интервал (мкс)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Сравнение PID-exploration vs НС-контроллер

In [ ]:
pid_df = env_df[env_df['global_step'] <= exploration_steps]
nn_df = env_df[env_df['global_step'] > exploration_steps]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(pid_df['error'], bins=100, alpha=0.7, color='tab:blue', edgecolor='black', linewidth=0.3)
axes[0].set_title(f'Распределение ошибки: PID (шаги 1–{exploration_steps})')
axes[0].set_xlabel('Error')
axes[0].set_ylabel('Частота')
axes[0].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[0].grid(True, alpha=0.3)

axes[1].hist(nn_df['error'], bins=100, alpha=0.7, color='tab:orange', edgecolor='black', linewidth=0.3)
axes[1].set_title(f'Распределение ошибки: НС (шаги {exploration_steps + 1}–{env_df["global_step"].max()})')
axes[1].set_xlabel('Error')
axes[1].set_ylabel('Частота')
axes[1].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"PID  — mean |error|: {pid_df['error'].abs().mean():.2f}, std: {pid_df['error'].std():.2f}, median: {pid_df['error'].median():.2f}")
print(f"НС   — mean |error|: {nn_df['error'].abs().mean():.2f}, std: {nn_df['error'].std():.2f}, median: {nn_df['error'].median():.2f}")

## Скользящее среднее ошибки

In [ ]:
alpha = 0.2
env_df['abs_error_ema'] = env_df['error'].abs().ewm(alpha=alpha, adjust=False).mean()

plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['abs_error_ema'], linewidth=1.0, color='tab:red',
         label=f'|Error| (EMA, α={alpha})')
plt.axvline(x=exploration_steps, color='gray', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Экспоненциальное скользящее среднее абсолютной ошибки')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('|Error| (EMA)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Анализ процесса обучения

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(train_df['step'], train_df['loss_q1'], alpha=0.6, linewidth=0.3, color='tab:blue', label='Q1 Loss')
plt.title('Critic Q1 Loss')
plt.xlabel('Шаг обучения')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(train_df['step'], train_df['loss_q2'], alpha=0.6, linewidth=0.3, color='tab:green', label='Q2 Loss')
plt.title('Critic Q2 Loss')
plt.xlabel('Шаг обучения')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(train_df['step'], train_df['alpha'], alpha=0.6, linewidth=0.3, color='tab:green')
plt.title('Alpha')
plt.xlabel('Шаг обучения')
plt.ylabel('Alpha')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
actor_df = train_df[train_df['actor_loss'].notna()]

plt.figure(figsize=(14, 5))
plt.plot(actor_df['step'], actor_df['actor_loss'], alpha=0.6, linewidth=0.3, color='tab:red', label='Actor Loss')
plt.title('Actor Loss')
plt.xlabel('Шаг обучения')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(train_df['step'], train_df['buffer_size'], linewidth=1.0, color='tab:purple')
plt.title('Размер буфера воспроизведения')
plt.xlabel('Шаг обучения')
plt.ylabel('Размер буфера')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Анализ логов соединения (phase_shifter)

Логи `event=exchange` из `collector.jsonl` — сырые обмены с фазовращателем через последовательный порт. Каждая запись содержит отправленный `control_output` и полученный `process_variable`. Эти данные включают все обмены, в том числе во время reset-фаз окружения.

In [ ]:
connection_df = load_jsonl(EXPERIMENT_DIR_PATH / "collector.jsonl", event="exchange")
connection_df = connection_df.reset_index(drop=True)
connection_df['global_step'] = connection_df.index + 1
print(f"Обмены с фазовращателем: {len(connection_df)} записей")
print(f"control_output: [{connection_df['control_output'].min()}, {connection_df['control_output'].max()}]")
print(f"process_variable: [{connection_df['process_variable'].min()}, {connection_df['process_variable'].max()}]")
connection_df.head()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(connection_df['global_step'], connection_df['process_variable'], alpha=0.7, linewidth=0.3, color='tab:blue')
plt.title('Process Variable (логи соединения)')
plt.xlabel('Номер обмена')
plt.ylabel('Process Variable')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(connection_df['global_step'], connection_df['control_output'], alpha=0.7, linewidth=0.3, color='tab:green')
plt.title('Control Output (логи соединения)')
plt.xlabel('Номер обмена')
plt.ylabel('Control Output')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(connection_df['global_step'], connection_df['control_output'], alpha=0.7, linewidth=0.3, color='tab:green')
axes[0].set_title('Control Output')
axes[0].set_ylabel('Control Output')
axes[0].grid(True, alpha=0.3)

axes[1].plot(connection_df['global_step'], connection_df['process_variable'], alpha=0.7, linewidth=0.3, color='tab:blue')
axes[1].set_title('Process Variable')
axes[1].set_xlabel('Номер обмена')
axes[1].set_ylabel('Process Variable')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Логи соединения: Control Output и Process Variable', fontsize=13)
plt.tight_layout()
plt.show()

## Анализ буфера воспроизведения

Загрузка сохранённого `ReplayBuffer` и анализ распределений наблюдений, действий и наград.

In [ ]:
from nn_laser_stabilizer.rl.data.replay_buffer import ReplayBuffer

buffer = ReplayBuffer.load(EXPERIMENT_DIR_PATH / "data" / "replay_buffer.pth")
n = buffer.size
print(f"Размер буфера: {n}")
print(f"Ёмкость: {buffer.capacity}")
print(f"obs_dim: {buffer.observations.shape[1]}, action_dim: {buffer.actions.shape[1]}")

buf_obs = buffer.observations[:n].numpy()       # (N, 3): error, error_prev, error_prev_prev
buf_actions = buffer.actions[:n].numpy()         # (N, 1): delta_norm
buf_rewards = buffer.rewards[:n].numpy()         # (N, 1)
buf_next_obs = buffer.next_observations[:n].numpy()
buf_dones = buffer.dones[:n].numpy()

print(f"\nСтатистика наблюдений (нормализованные):")
for i, name in enumerate(['error', 'error_prev', 'error_prev_prev']):
    print(f"  {name}: mean={buf_obs[:, i].mean():.4f}, std={buf_obs[:, i].std():.4f}, "
          f"min={buf_obs[:, i].min():.4f}, max={buf_obs[:, i].max():.4f}")

print(f"\nСтатистика действий:")
print(f"  delta_norm: mean={buf_actions.mean():.4f}, std={buf_actions.std():.4f}, "
      f"min={buf_actions.min():.4f}, max={buf_actions.max():.4f}")

print(f"\nСтатистика наград:")
print(f"  reward: mean={buf_rewards.mean():.4f}, std={buf_rewards.std():.4f}, "
      f"min={buf_rewards.min():.4f}, max={buf_rewards.max():.4f}")

print(f"\nDones: {buf_dones.sum()} / {n} ({buf_dones.mean() * 100:.2f}%)")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].hist(buf_obs[:, 0], bins=100, alpha=0.7, color='tab:blue', edgecolor='black', linewidth=0.3)
axes[0, 0].set_title('error (norm)')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].hist(buf_obs[:, 1], bins=100, alpha=0.7, color='tab:cyan', edgecolor='black', linewidth=0.3)
axes[0, 1].set_title('error_prev (norm)')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].hist(buf_actions.flatten(), bins=100, alpha=0.7, color='tab:purple', edgecolor='black', linewidth=0.3)
axes[1, 0].set_title('action (delta_norm)')
axes[1, 0].set_xlabel('Значение')
axes[1, 0].set_ylabel('Частота')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(buf_rewards.flatten(), bins=100, alpha=0.7, color='tab:green', edgecolor='black', linewidth=0.3)
axes[1, 1].set_title('reward')
axes[1, 1].set_xlabel('Значение')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Распределения данных в буфере воспроизведения', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(buf_obs[:, 0], buf_actions.flatten(), alpha=0.05, s=1, color='tab:purple')
plt.xlabel('error (norm)')
plt.ylabel('action (delta_norm)')
plt.title('Буфер: зависимость action от error')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist2d(buf_obs[:, 0], buf_obs[:, 1], bins=100, cmap='hot_r')
plt.colorbar(label='Количество переходов')
plt.xlabel('error (norm)')
plt.ylabel('error_prev (norm)')
plt.title('Покрытие пространства состояний в буфере')
plt.tight_layout()
plt.show()